Это оригинальный код.

In [ ]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))
pd.set_option('display.max_colwidth', 1000)
​
PATH_TO_DATA = 'D:\pycharm\KeyWordExtraction\\venv\data'
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]
data = pd.concat([pd.read_json(file, lines=True, encoding='utf-8') for file in files][:1], axis=0, ignore_index=True)
​
​
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
​
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
​
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
​
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
​
        if (tp + fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
​
        if (tp + fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec + rec) == 0:
            f1 = 0
        else:
            f1 = (2 * (prec * rec)) / (prec + rec)
​
        jac = tp / union
​
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))
​
​
from string import punctuation
from nltk.corpus import stopwords
​
punct = punctuation + '«»—…“”*№–'
stops = set(stopwords.words('russian'))
​
def normalize(text):
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']
​
    return words
​
data['content_norm'] = data['content'].apply(normalize)
data['title_norm'] = data['title'].apply(normalize)
data['content_norm_str'] = data['content_norm'].apply(' '.join)
​
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)
tfidf.fit(data['content_norm_str'])
​
​
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors = tfidf.transform(data['content_norm_str'])
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]]

Первый мотод: больше нормализации. В результате есть имя "том", которое вероятно не будет ключевым словом. Пожтому я сделал список 5000 имен на русском языке и удалил из результаты имена есть они в сптслк имен. 

In [ ]:
with open('names.txt','r',encoding='utf-8') as names:
    names = names.read()
    names = names.lower()
    names = names.split('\n')
​
for entry in keywords:
    for word in entry:
        if entry in names:
            entry.remove(word)
​
evaluate(data['keywords'], keywords)

In [ ]:
for entry in keywords:
    for word_1 in entry:
        for word_2 in entry:
            if word_1 + ' ' + word_2 in entry or word_2 + ' ' + word_1 in entry:
​
                try:
                    entry.remove(word_1)
                except:
                    pass
                try:
                    entry.remove(word_2)
                except:
                    pass
​
​
​
            if word_1 != word_2:
                if word_1 != word_2:
                    try:
                        entry.remove(word_1)
                    except:
                        pass
                    
evaluate(data['keywords'], keywords)

In [ ]:
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-6:-1]]

In [ ]:
with open('results.txt','r',encoding='utf-8') as results:
    results = results.read()
    results = results.split('\n')
    for entry in keywords:
        for word in entry:
            if word not in results:
                entry.remove(word)
                
evaluate(data['keywords'], keywords)

In [ ]:
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-7:-1]]